In [1]:
import itertools
import jsonlines

In [2]:
from datasets import load_dataset
from pprint import pprint

In [3]:
#llama can't be installed directly, 
#so run: pip install lamini, which will install llama package then run the following code

from llama import BasicModelRunner

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [5]:
instruction_tuned_dataset = load_dataset("tatsu-lab/alpaca", split="train", streaming=True)

In [6]:
m = 5
top_m = itertools.islice(instruction_tuned_dataset, m)
print("Instruction tuned dataset...")
for x in top_m:
    pprint(x)

Instruction tuned dataset...
{'input': '',
 'instruction': 'Give three tips for staying healthy.',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': 'Below is an instruction that describes a task. Write a response that '
         'appropriately completes the request.\n'
         '\n'
         '### Instruction:\n'
         'Give three tips for staying healthy.\n'
         '\n'
         '### Response:\n'
         '1.Eat a balanced diet and make sure to include plenty of fruits and '
         'vegetables. \n'
         '2. Exercise regularly to keep your body active and strong. \n'
         '3. Get enough sleep and maintain a consistent sleep schedule.'}
{'input': '',
 'instruction': 'What are the three primary colors?',
 'output': 'The three primary colors are red, blue, a

In [7]:
#two prompt templates

#1. with input
prompt_template_with_input = """Below is an instruction that describes a task, paired with an input that provides further context.
Write response that completes the request as per the instruction.
###Instruction:
{instruction}

###Input:
{input}

###Response:"""

#without input

prompt_template_without_input="""Below is an instruction that describes a task.
Write response that completes the request as per the instruction.
###Instruction:
{instruction}

###Response:"""

In [ ]:
m=10000
top_m = itertools.islice(instruction_tuned_dataset, m)
processed_data = []
for j in top_m:
    #without input
    if not j['input']:         
        processed_prompt = prompt_template_without_input.format(instruction=j['instruction'])
    #with input
    else:
        processed_prompt = prompt_template_with_input.format(instruction=j['instruction'], input=j['input'])
    processed_data.append({'input':processed_prompt, 'output':j['output']})
        

In [ ]:
pprint(processed_data[0])

In [ ]:
#save data
with jsonlines.open(f'alpaca_processed_10000.jsonl', 'w') as writer:
    writer.write_all(processed_data)

In [ ]:
#load data from huggingface 


In [8]:
non_instruct_model = BasicModelRunner('meta-llama/Llama-2-7b-hf')

In [ ]:
non_instruct_output = non_instruct_model("Tell me how to train my dog")
print("Non instruction-tuned model (Llama-2-7b-hf)'s output:", non_instruct_output)

In [11]:
from transformers import TFAutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")

In [13]:
from datasets import load_dataset

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [15]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
    #tokenize
    input_ids = tokenizer.encode(
        text, return_tensors='pt',
        truncation=True,
        max_length=max_output_tokens
    )

    #generate
    device = model.device
    generated_tokens_with_prompt = model.generate(
        input_ids = input_ids.to(device),
        max_length = max_output_tokens
    )
    #decode
    generated_text_with_prompt = tokenizer.batch_decode(
        generated_tokens_with_prompt, 
        skip_special_tokens=True
        )
    #strip the prompt
    generated_text_answer = generated_text_with_prompt[0][len(text):]

    return generated_text_answer


In [16]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")

In [17]:
question='Can Lamini generate technical documentation or user manuals for software projects?'

In [18]:
result = inference(question, model, tokenizer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [20]:
pprint(result)

('\n'
 '\n'
 'I have a question about the following:\n'
 '\n'
 'How do I get the correct documentation to work?\n'
 '\n'
 'A:\n'
 '\n'
 'I think you need to use the following code:\n'
 '\n'
 'A:\n'
 '\n'
 'You can use the following code to get the correct documentation.\n'
 '\n'
 'A:\n'
 '\n'
 'You can use the following code to get the correct documentation.\n'
 '\n'
 'A:\n'
 '\n'
 'You can use the following')


In [21]:
instruction_finetuned_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")

In [22]:
pprint(inference(question, instruction_finetuned_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


('Yes, Lamini can generate technical documentation or user manuals for '
 'software projects. This can be achieved by providing a prompt for a specific '
 'technical question or question to the LLM Engine, or by providing a prompt '
 'for a specific technical question or question. Additionally, Lamini can be '
 'trained on specific technical questions or questions to help users '
 'understand the process and provide feedback to the LLM Engine. Additionally, '
 'Lamini')
